#  COVID-19 Data Visualization 35 Days After the first case

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,mean_absolute_error
import datetime
import operator
plt.style.use('seaborn')
%matplotlib inline

## Using Ben Baldwin´s code: 
code can be found here:
https://github.com/guga31bb/nfl/blob/master/code/COVID_Python_Starter.ipynb

## Number of deaths by COVID-19 in the first 35 days:

In [47]:
from plotly import graph_objs as go
pd.set_option('mode.chained_assignment', None)

In [48]:
deaths=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [49]:
deaths2 = pd.melt(deaths,id_vars=['Province/State','Country/Region','Lat','Long'])
deaths2.rename(columns={"variable": "date", "value": "deaths"},inplace=True)

In [50]:
deaths2['date'] = pd.to_datetime(deaths2['date'])
deaths_agg = pd.DataFrame(deaths2.groupby(['Country/Region','date'])['deaths'].sum()).reset_index()

In [51]:
countries_of_interest = ['Brazil','Argentina','Italy','Spain','France','Korea, South','Mexico','US']
final = deaths_agg[deaths_agg['Country/Region'].isin(countries_of_interest)]


In [52]:
final['d1'] = np.where(final['deaths']>= 1,1,0)
final['days_since_1']=final.groupby(by=['Country/Region'])['d1'].cumsum()


In [53]:
#LIMIT TO ONLY FIRST 35 DAYS SINCE 10 DEATHS
final_use = final[final['days_since_1']<=35]

print("The most recent day is", np.max(final_use['date']))

The most recent day is 2020-04-08 00:00:00


In [54]:
display(final_use)

,Country/Region,date,deaths,d1,days_since_1
468,Argentina,2020-01-22,0,0,0
469,Argentina,2020-01-23,0,0,0
470,Argentina,2020-01-24,0,0,0
471,Argentina,2020-01-25,0,0,0
472,Argentina,2020-01-26,0,0,0
473,Argentina,2020-01-27,0,0,0
474,Argentina,2020-01-28,0,0,0
475,Argentina,2020-01-29,0,0,0
476,Argentina,2020-01-30,0,0,0
477,Argentina,2020-01-31,0,0,0


In [55]:
fig = go.Figure()
annotations = []
i = 0

for c in countries_of_interest:
    color = px.colors.qualitative.G10[i]
    annotations.append(go.layout.Annotation(
        x = final_use[final_use['Country/Region']=='{}'.format(c)].iloc[-1]['days_since_1'],
        y = final_use[final_use['Country/Region']=='{}'.format(c)].iloc[-1]['deaths'],
        text=c,
        font=dict(
            family="Franklin Gothic",
            size=16,
            color="black"
            ))
        )

    #make the data in a bad way
    tmp = final_use[final_use['Country/Region'].isin([c])]
    y1= tmp['deaths']
    x1 = tmp['days_since_1']

    #this lets us control the lines
    fig.add_trace(go.Scatter(x=x1, y=y1, name = c, mode='lines+markers', line=dict(color = color, width=3)))
    i += 1

fig.update(layout=go.Layout(showlegend=False, annotations=annotations, height=500, width=1000))
fig.update_layout(
    title={
      'text': "COVID-19 tracker: Cumulative Deaths",
      'y':0.9,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'},
    xaxis_title="Days since first death"
)

max = np.max(final_use['date'])
note = 'Updated ' + str(max.month) + '-' + str(max.day)
fig.add_annotation(
        x=5,
        y=9000,
        xref="x",
        yref="y",
        text=note,
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="red"
            ),
        align="center",
        ax=20,
        ay=-30,
        bordercolor="black",
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
        )

fig.add_annotation(
    x=5,
    y=8000,
    text="Source: JHU",
    showarrow=False,
    font=dict(
      family="Courier New, monospace",
      size=16,
      color="red"
      ),
)


#print(fig.layout)
fig.layout.template = 'plotly_white'
fig.layout.template = 'presentation'

fig.show()

## COVID-19 cases after the first 35 days



In [56]:
cases=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [57]:
confirmed2 = pd.melt(cases,id_vars=['Province/State','Country/Region','Lat','Long'])
confirmed2.rename(columns={"variable": "date", "value": "cases"},inplace=True)

In [58]:
confirmed2['date'] = pd.to_datetime(confirmed2['date'])
confirmed_agg = pd.DataFrame(confirmed2.groupby(['Country/Region','date'])['cases'].sum()).reset_index()

In [59]:
countries_of_interest2 = ['Brazil','Italy','Spain','France','Mexico','US','Korea, South','Chile','Colombia','Ecuador']
final2 = confirmed_agg[confirmed_agg['Country/Region'].isin(countries_of_interest2)]

In [60]:
#Adjust number of cases
final2['d1'] = np.where(final2['cases']>= 1,1,0)
final2['days_since_1']=final2.groupby(by=['Country/Region'])['d1'].cumsum()

In [61]:
#LIMIT TO ONLY FIRST 35 DAYS SINCE FIRST CONFIRMED CASE
final_use2 = final2[final2['days_since_1']<=35]

print("The most recent day is", np.max(final_use2['date']))

The most recent day is 2020-04-08 00:00:00


In [62]:
display(final_use2)

,Country/Region,date,cases,d1,days_since_1
1794,Brazil,2020-01-22,0,0,0
1795,Brazil,2020-01-23,0,0,0
1796,Brazil,2020-01-24,0,0,0
1797,Brazil,2020-01-25,0,0,0
1798,Brazil,2020-01-26,0,0,0
1799,Brazil,2020-01-27,0,0,0
1800,Brazil,2020-01-28,0,0,0
1801,Brazil,2020-01-29,0,0,0
1802,Brazil,2020-01-30,0,0,0
1803,Brazil,2020-01-31,0,0,0


In [63]:
fig = go.Figure()
annotations = []
i = 0

for c in countries_of_interest2:
    color = px.colors.qualitative.G10[i]
    annotations.append(go.layout.Annotation(
        x = final_use2[final_use2['Country/Region']=='{}'.format(c)].iloc[-1]['days_since_1'],
        y = final_use2[final_use2['Country/Region']=='{}'.format(c)].iloc[-1]['cases'],
        text=c,
        font=dict(
            family="Franklin Gothic",
            size=16,
            color="black"
            ))
        )
    

    #make the data in a bad way
    tmp = final_use2[final_use2['Country/Region'].isin([c])]
    y1= tmp['cases']
    x1 = tmp['days_since_1']

    #this lets us control the lines
    fig.add_trace(go.Scatter(x=x1, y=y1, name = c, mode='lines+markers', line=dict(color = color, width=2),marker=dict(size=7)))
    i += 1

fig.update(layout=go.Layout(showlegend=False, annotations=annotations, height=600, width=1000))
fig.update_layout(
    title={
      'text': "COVID-19 Tracker: Confirmed cases since first case",
      'y':0.9,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'},
    xaxis_title="Days since first case",
    yaxis_title="Cases"
)

max = np.max(final_use2['date'])
note = 'Updated ' + str(max.month) + '-' + str(max.day)
fig.add_annotation(
        x=5,
        y=9000,
        xref="x",
        yref="y",
        text=note,
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="red"
            ),
        align="center",
        ax=20,
        ay=-30,
        bordercolor="black",
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
        )

fig.add_annotation(
    x=5,
    y=8000,
    text="Source: JHU",
    showarrow=False,
    font=dict(
      family="Courier New, monospace",
      size=16,
      color="red"
      ),
)
#print(fig.layout)
fig.update_layout(legend=dict(x=1.2, y=0))
fig.layout.template = 'plotly_dark'
fig.layout.template = 'presentation'
fig.show()

In [28]:
cases35= final_use2.groupby(['Country/Region'],as_index=False,sort=True)['cases'].max().sort_values(by='cases').reset_index(drop=True)
cases35.rename(columns={'cases':'# of cases after 35 days'})
#cases35.replace({'Chile': 'Chile*', 'Ecuador': "Ecuador*",'Colombia':'Colombia*'})
j=cases35[cases35['Country/Region']=="Colombia"].index
final=cases35.drop(j)
i=final[final['Country/Region']=="Chile"].index
final2=final.drop(i).reset_index(drop=True).rename(columns={'cases':'# of cases after 35 days'})
final2


,Country/Region,# of cases after 35 days
0,France,38
1,US,51
2,Spain,400
3,"Korea, South",977
4,Mexico,1378
5,Ecuador,3465
6,Italy,3858
7,Brazil,5717
